# Module 5: Advanced Indexing
**Goal**: Shatter the illusion that "Indexing Everything" makes databases faster.

In the previous chapter, we learned about the B-Tree—the "Telephone Book" of the database world. It is excellent for unique lookups.

But what if we aren't looking for a unique ID?
1. **What if we are filtering by Gender or Status?** (There are only a few distinct values).
2. **What if we are in the Cloud?** (Where B-Trees are too heavy to maintain).
3. What does an index cost?** (We pay for read speed with write slowness).

In this lab, we will expose the physics of **Bitmaps**, **Write Amplification**, and **Zone Maps**.

---

## 1. Setup and Data Preparation
First, we establish our connections and prepare the specific files needed for the Zone Map experiment. We will convert our CSVs to Parquet to demonstrate how column stores use metadata.

In [ ]:
import pandas as pd
import duckdb
import psycopg2
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Connect to the embedded OLAP engine (DuckDB)
con_duck = duckdb.connect(database=':memory:')

# Connect to the OLTP engine (Postgres)
# Note: Ensure your Docker container matches these credentials
db_params = {
    "host": "db_int_opt",
    "port": 5432,
    "user": "admin",
    "password": "password",
    "dbname": "db_int_opt"
}

def get_pg_connection():
    conn = psycopg2.connect(**db_params)
    conn.autocommit = True
    return conn

# PREP WORK: Convert CSVs to Parquet for Section 5.3
# We need a Sorted file and a Shuffled file to prove how Zone Maps work.
print("Converting CSVs to Parquet for the Lab...")

con_duck.execute("""
    CREATE OR REPLACE TABLE orders_sorted AS SELECT * FROM read_csv_auto('../data/orders_sorted.csv');
    COPY orders_sorted TO '../data/orders_sorted.parquet' (FORMAT PARQUET);
    
    CREATE OR REPLACE TABLE orders_shuffled AS SELECT * FROM read_csv_auto('../data/orders_shuffled.csv');
    COPY orders_shuffled TO '../data/orders_shuffled.parquet' (FORMAT PARQUET);
""")

print("Setup Complete. Data is ready.")

---

## 5.1 Bitmap Operations: The Power of Low Cardinality
**The Physical Concept**: Standard B-Trees struggle when a column has very Low Cardinality (few unique values, like "Status: Active/Inactive"). If 50% of your users are "Active," a B-Tree is useless because it would still return half the table.

Instead, databases use **Bitmaps**. Imagine a simple string of bits: `10011`.
- User 1 (Active) -> 1
- User 2 (Inactive) -> 0
- User 3 (Inactive) -> 0

Computers can process bitwise operations (AND/OR) on these streams incredibly fast—much faster than comparing Integers or Strings.

**The Experiment**: We will use DuckDB (a Column Store that relies heavily on vectorization and compression similar to Bitmaps) to compare the speed of filtering on a Low Cardinality column vs. a High Cardinality column.

#### Step 1: Hypothesis
"Will filtering by a Boolean (Low Cardinality) be faster than filtering by a specific User ID (High Cardinality) even if they return the same amount of data?"

### Step 2: The Code

In [ ]:
# Load the 100k Users data
con_duck.execute("CREATE OR REPLACE TABLE users AS SELECT * FROM read_parquet('../data/users.parquet')")

# We want to select approximately the same number of rows to make it a fair fight.
# Let's see how many users are 'Active' (Low Cardinality)
count = con_duck.execute("SELECT COUNT(*) FROM users WHERE account_status = 'Active'").fetchone()[0]
print(f"Target Row Count: {count}")

# 1. Low Cardinality Filter (Account Status)
start_low = time.time()
for _ in range(50): # Run 50 times to amplify the CPU effect
    con_duck.execute("SELECT count(*) FROM users WHERE account_status = 'Active'").fetchall()
end_low = time.time()
avg_low = (end_low - start_low) / 50

# 2. High Cardinality Filter (User ID)
# We filter for a range of IDs that roughly equals the count of 'Active' users to simulate scanning
start_high = time.time()
for _ in range(50):
    con_duck.execute(f"SELECT count(*) FROM users WHERE user_id < {count}").fetchall()
end_high = time.time()
avg_high = (end_high - start_high) / 50

print(f"Low Cardinality Time: {avg_low:.6f}s")
print(f"High Cardinality Time: {avg_high:.6f}s")

#### Step 3: Visualization

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(['Low Cardinality (Status)', 'High Cardinality (ID)'], [avg_low, avg_high], color=['green', 'red'])
plt.title('Filter Performance: Bitwise vs. Integer Comparison')
plt.ylabel('Execution Time (seconds)')
plt.show()

#### Step 4: Analysis
**Why was the Low Cardinality query faster?** In a column store, low cardinality data is often stored using Dictionary Encoding or Run-Length Encoding. The engine doesn't scan the text "Active"; it scans a compressed list of small integers or bits.
- CPU Efficiency: The CPU can process 64 "Active/Inactive" flags in a single cycle using SIMD (Single Instruction, Multiple Data) instructions.
- High Cardinality: Comparing unique Integers (`user_id < 50000`) requires more complex logic for every single row, consuming more CPU cycles.

----

## 5.2 The Cost of Indexes: Write Amplification
**The Physical Concept**: New engineers often say, "My query is slow? I'll just add an index!" But Indexes are not free magic. An index is a copy of your data, sorted in a specific way. Every time you `INSERT` a row into a table, the database must also `INSERT` that entry into every single index attached to that table. This is called Write Amplification.

**The Experiment**: We will use Postgres (OLTP) to insert 50,000 rows into two tables:
1. Light Table: 0 Indexes.
2. Heavy Table: 4 Indexes.

#### Step 1: Hypothesis
"How much slower will the Heavy Table be? 2x? 10x?"

#### Step 2: The Code

In [ ]:
import io

# 5.2 Final: The "Firehose" Test (COPY Protocol)

pg_conn = get_pg_connection()
pg_conn.autocommit = True # We want raw speed, let the DB manage internal locking
cur = pg_conn.cursor()

# 1. Reset Tables
cur.execute("DROP TABLE IF EXISTS write_test_light;")
cur.execute("CREATE TABLE write_test_light (id SERIAL PRIMARY KEY, uuid TEXT, data TEXT, created_at TIMESTAMP);")

cur.execute("DROP TABLE IF EXISTS write_test_heavy;")
cur.execute("CREATE TABLE write_test_heavy (id SERIAL PRIMARY KEY, uuid TEXT, data TEXT, created_at TIMESTAMP);")

# 2. Add 4 Indexes to the Heavy table
cur.execute("CREATE INDEX idx_heavy_uuid ON write_test_heavy(uuid);")
cur.execute("CREATE INDEX idx_heavy_data ON write_test_heavy(data);")
cur.execute("CREATE INDEX idx_heavy_created ON write_test_heavy(created_at);")
cur.execute("CREATE INDEX idx_heavy_composite ON write_test_heavy(uuid, created_at);")

# 3. Prepare Data in Memory (CSV Format)
# We use 100k rows now because COPY is so fast, 50k might be too quick to measure!
print("Generating 100k rows of CSV data in memory...")
csv_buffer = io.StringIO()
# Format: uuid, data, created_at (id is auto-generated so we skip it in copy)
for x in range(100000):
    csv_buffer.write(f"uuid-{x}\tdata-{x}\t2023-01-01\n")
csv_buffer.seek(0)

print("Starting Firehose (COPY)...")

# 4. Measure Write Speed: Light Table
start_light = time.time()
csv_buffer.seek(0) # Rewind buffer
# COPY is atomic and optimized for throughput
cur.copy_from(csv_buffer, 'write_test_light', columns=('uuid', 'data', 'created_at'))
end_light = time.time()

# 5. Measure Write Speed: Heavy Table
start_heavy = time.time()
csv_buffer.seek(0) # Rewind buffer
cur.copy_from(csv_buffer, 'write_test_heavy', columns=('uuid', 'data', 'created_at'))
end_heavy = time.time()

pg_conn.close()

time_light = end_light - start_light
time_heavy = end_heavy - start_heavy

print(f"Light Table (No Index): {time_light:.4f}s")
print(f"Heavy Table (4 Indexes): {time_heavy:.4f}s")
print(f"Write Amplification Factor: {time_heavy / time_light:.1f}x Slower")

#### Step 3: Visualization

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=['No Extra Indexes', '4 Extra Indexes'], y=[time_light, time_heavy], palette='magma')
plt.title('The Cost of Indexes: Write Amplification')
plt.ylabel('Time to Insert 50k Rows (s)')
plt.show()

#### Step 4: Analysis
You likely saw a massive slowdown (often 5x-10x).
- **Physics**: When you wrote to write_test_light, Postgres appended data to the Heap (the main storage). Simple.
- **The Burden**: When you wrote to write_test_heavy, Postgres had to:
    1. Write to the Heap.
    2. Traverse the B-Tree for `uuid` to find the leaf node.
    3. Traverse the B-Tree for `data`.
    4. Traverse the B-Tree for `created_at`.
    5. Traverse the B-Tree for the composite index.
- **Page Splits**: If an index page was full, Postgres had to split the page, move data around, and rebalance the tree. This is pure I/O overhead.

---